# Imports

In [1]:
import pandas as pd
from apyori import apriori

# Montagem da base de dados

## Leitura da tabela

In [ ]:
dados_base = pd.read_csv('Data/MercadoSim.csv', sep=';',encoding='cp1252', header=None)
# Mudança das colunas para refletir o nome do produto
produtos = ['Leite', 'Cafe', 'Cerveja', 'Pao', 'Manteiga', 'Arroz', 'Feijao']
dados_base.columns = produtos
#
dados_base

,0,1,2,3,4,5,6
0,NaN,Cafe,NaN,Pao,Manteiga,NaN,NaN
1,Leite,NaN,Cerveja,Pao,Manteiga,NaN,NaN
2,NaN,Cafe,NaN,Pao,Manteiga,NaN,NaN
3,Leite,Cafe,NaN,Pao,Manteiga,NaN,NaN
4,NaN,NaN,Cerveja,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,Manteiga,NaN,NaN
6,NaN,NaN,NaN,Pao,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,Feijao
8,NaN,NaN,NaN,NaN,NaN,Arroz,Feijao
9,NaN,NaN,NaN,NaN,NaN,Arroz,NaN


In [3]:
# Dimensões da tabela de dados
dados_base.shape

(10, 7)

## Transformação em conjunto de listas

In [ ]:
# O método do algoritmo apriori necessita de listas como input
transacoes = []
for i in range(len(dados_base)):
  transacoes.append([str(dados_base.values[i, j]) for j in range(dados_base.shape[1])])
#
transacoes

[['nan', 'Cafe', 'nan', 'Pao', 'Manteiga', 'nan', 'nan'],
 ['Leite', 'nan', 'Cerveja', 'Pao', 'Manteiga', 'nan', 'nan'],
 ['nan', 'Cafe', 'nan', 'Pao', 'Manteiga', 'nan', 'nan'],
 ['Leite', 'Cafe', 'nan', 'Pao', 'Manteiga', 'nan', 'nan'],
 ['nan', 'nan', 'Cerveja', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'Manteiga', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'Pao', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'Feijao'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'Arroz', 'Feijao'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'Arroz', 'nan']]

In [5]:
type(transacoes)

list

# Chamada do algoritmo

## Parâmetros

In [ ]:
suporte_min = 0.3   # Suporte mínimo dos datasets
confianca_min = 0.8 # Confiança mínima das regras

## Algoritmo

In [ ]:
regras = apriori(transactions= transacoes, min_support= suporte_min, min_confidence= confianca_min)
saida = list(regras)
print(len(saida))
print(saida)

# Formato base de cada item em 'saida'
# item[1] = suporte
# item[2] = regras possíveis
#   regra[0] = antecedente
#   regra[1] = consequente
#   regra[2] = confiança
#   regra[3] = lift

12
[RelationRecord(items=frozenset({'nan'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'nan'}), confidence=1.0, lift=1.0)]), RelationRecord(items=frozenset({'Manteiga', 'Cafe'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Cafe'}), items_add=frozenset({'Manteiga'}), confidence=1.0, lift=2.0)]), RelationRecord(items=frozenset({'Pao', 'Cafe'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Cafe'}), items_add=frozenset({'Pao'}), confidence=1.0, lift=2.0)]), RelationRecord(items=frozenset({'nan', 'Cafe'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Cafe'}), items_add=frozenset({'nan'}), confidence=1.0, lift=1.0)]), RelationRecord(items=frozenset({'Pao', 'Manteiga'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Manteiga'}), items_add=frozenset({'Pao'}), confidence=0.8, lift=1.6), OrderedStatistic(items_base=frozenset({'Pao'})

# Transformação em tabela/matriz/dataframe

In [ ]:
# Variáveis de lista de suporte
Antecedente = []; Consequente = []
suporte = []; confianca = []; lift = []
RegrasFinais = []

# Para cada instância em saida
for resultado in saida:
  result_suporte = resultado[1] # Suporte
  result_rules = resultado[2] # Regras possíveis de cada combinação
  # Para cada regra possível
  for result_rule in result_rules:
    result_antecedente = list(result_rule[0]) # Antecedente da regra
    result_consequente = list(result_rule[1]) # Consequente da regra
    result_confianca = result_rule[2] # Confiança da regra
    result_lift = result_rule[3] # Lift da regra

    # Filtrar se algum dos elementos for nulo ou se não houver elementos
    if 'nan' in result_antecedente or 'nan' in result_consequente: continue
    if len(result_antecedente) == 0 or len(result_consequente) == 0: continue

    Antecedente.append(result_antecedente)
    Consequente.append(result_consequente)
    suporte.append(result_suporte)
    confianca.append(result_confianca)
    lift.append(result_lift)

# Montar tabela com os arrays montados
RegrasFinais = pd.DataFrame({'Antecedente': Antecedente, 'Consequente': Consequente, 'suporte': suporte, 'confianca': confianca, 'lift': lift})

In [14]:
RegrasFinais

,Antecedente,Consequente,suporte,confianca,lift
0,[Cafe],[Manteiga],0.3,1.0,2.0
1,[Cafe],[Pao],0.3,1.0,2.0
2,[Manteiga],[Pao],0.4,0.8,1.6
3,[Pao],[Manteiga],0.4,0.8,1.6
4,[Cafe],"[Pao, Manteiga]",0.3,1.0,2.5
5,"[Manteiga, Cafe]",[Pao],0.3,1.0,2.0
6,"[Pao, Cafe]",[Manteiga],0.3,1.0,2.0


## Ordenação das regras

In [ ]:
# Ordenação por lft
RegrasFinais.sort_values(by='lift', ascending =False)

,Antecedente,Consequente,suporte,confianca,lift
4,[Cafe],"[Pao, Manteiga]",0.3,1.0,2.5
1,[Cafe],[Pao],0.3,1.0,2.0
0,[Cafe],[Manteiga],0.3,1.0,2.0
6,"[Pao, Cafe]",[Manteiga],0.3,1.0,2.0
5,"[Manteiga, Cafe]",[Pao],0.3,1.0,2.0
2,[Manteiga],[Pao],0.4,0.8,1.6
3,[Pao],[Manteiga],0.4,0.8,1.6
